In [3]:
import numpy as np
import datetime
import copy
np.seterr(divide='raise', invalid='raise')

{'divide': 'raise', 'invalid': 'raise', 'over': 'warn', 'under': 'ignore'}

In [4]:
def generateBinaryData(n, T, R_scale = 1):
    L = np.random.randint(low=0, high=2, size=(n,T))
    R = np.zeros((n,T))
    means = np.random.random_sample() * R_scale
    stds = np.random.random_sample()
    for i in range(T):
        R[:,i] = np.random.logistic(loc=means, scale=stds, size=(n,))
        R[:,i] = np.where(R[:,i] > 0, 1, 0)
        means = np.mean(R[:,:i+1] + L[:,:i+1])
        stds = np.std(R[:,:i+1] + L[:,:i+1])
    print(np.count_nonzero(R[:,0]))
    
    L_indicator = (R != 0)
    obsL = np.where(L_indicator, L, float('nan'))
    return obsL

In [51]:
def initParams(data, numCats, numPatterns, n, T):
    params = {'theta':np.zeros(numCats), 'n': n, 'T': T, 'ns':np.zeros([numPatterns] + list(numCats[:T])),
              'pattern_ns': {}, 'phis': {}, 'binary_patterns': {}, 'numCatsTStep': numCats}
    old_params = {'theta':np.zeros(numCats), 'n': n, 'T': T, 'ns':np.zeros([numPatterns] + list(numCats[:T])),
              'pattern_ns': {}, 'phis': {}, 'binary_patterns': {}, 'numCatsTStep': numCats}
    ncomplete = np.zeros(numCats)
    
    for i in range(n):
        fullyObserved = True
        for j in range(T):
            if(np.isnan(data[i,j])):
                fullyObserved = False
        if(fullyObserved):
            ncomplete[tuple(data[i,:].astype(int))] += 1
            
    for idx, val in np.ndenumerate(params['theta']):
        params['theta'][idx] = ncomplete[idx] / n
        
    for i in range(1,T):
        params['phis'][i] = {}
        for numPhis in range(np.prod(numCats[:i])):
            params['phis'][i][numPhis] = np.random.random_sample(tuple(numCats[:i]))
            
    for k in range(numPatterns):
        params['binary_patterns'][k] = [int(x) for x in bin(k)[2:]]
        if (len(params['binary_patterns'][k]) < T):
            params['binary_patterns'][k] = np.array([0]*(T-len(params['binary_patterns'][k])) + params['binary_patterns'][k])
        else:
            params['binary_patterns'][k] = np.array(params['binary_patterns'][k])
    
    for k in range(numPatterns):
        params['pattern_ns'][k] = np.zeros(numCats[np.where(params['binary_patterns'][k] == 0)[0]])
        for i in range(n):
            for idx, val in np.ndenumerate(params['theta']):
                match = True
                for j in range(T):
                    if params['binary_patterns'][k][j] == 1:
                        if(not np.isnan(data[i,j])):
                            match = False
                            break
                    else:
                        if(data[i,j] != idx[j]):
                            match = False
                            break
                if(match):
                    if(k < numPatterns - 1):
                        params['pattern_ns'][k][tuple(np.array(idx)[np.where(params['binary_patterns'][k] == 0)[0]])] += 1
                    else:
                        params['pattern_ns'][k] += 1
                    break
    
    return params, old_params

def bin2dec(l):
    num = 0
    for i in range(len(l)):
        num += l[i] * (2 ** (len(l)-i-1))
    return num

In [52]:
def MStep(T, params):
    for idx, val in np.ndenumerate(params['theta']):
        entry = 0
        for n in range(params['ns'].shape[0]):
            entry += params['ns'][n][idx] / params['n']
        params['theta'][idx] = entry

    for t in range(1,T):
        for pat in range(np.prod(params['numCatsTStep'][:t])):
            for idx, val in np.ndenumerate(params['phis'][t][pat]):
                numerator = 0
                num_bin_rep = [int(x) for x in bin(pat)[2:]] + [1]
                if len(num_bin_rep) == T:
                    n = bin2dec(num_bin_rep)
                    numerator = params['ns'][n][idx]
                else:
                    for i in range(int(2 ** (T-len(num_bin_rep)))):
                        additional_bin = [int(x) for x in bin(i)[2:]]
                        n = bin2dec(num_bin_rep + additional_bin)
                        numerator += params['ns'][n][idx]

                denominator = 0
                den_bin_rep = [int(x) for x in bin(pat)[2:]]
                for i in range(int(2 ** (T-len(den_bin_rep)))):
                        additional_bin = [int(x) for x in bin(i)[2:]]
                        n = bin2dec(den_bin_rep + additional_bin)
                        denominator += params['ns'][n][idx]
            try:
                params['phis'][t][pat][idx] = np.sum(numerator) / np.sum(denominator) 
            except FloatingPointError: 
                params['phis'][t][pat][idx] = 0
                
    return params

def EStep(params):
    for n in range(params['ns'].shape[0]):
        for idx, val in np.ndenumerate(params['ns'][n]):
            numerator_prod = 1
            denominator_prod = 1
            observed = tuple(np.where(params['binary_patterns'][n] == 0, idx, slice(None)))
            num_unobserved = np.count_nonzero(params['binary_patterns'][n])
            for i in range(1,len(idx)):
                len_seq = i
                sub_pat = bin2dec(params['binary_patterns'][n][:i])
                L_idx_phi = idx[:i]
                sub_observed = tuple(np.where(params['binary_patterns'][n][:i] == 0, L_idx_phi, slice(None)))
                if params['binary_patterns'][n][i] == 0:
                    numerator_prod = params['phis'][len_seq][sub_pat][L_idx_phi] * numerator_prod
                    denominator_prod = params['phis'][len_seq][sub_pat][sub_observed].T * denominator_prod
                else:
                    numerator_prod = (1 - params['phis'][len_seq][sub_pat][L_idx_phi]) * numerator_prod
                    denominator_prod = (1 - params['phis'][len_seq][sub_pat][sub_observed]).T * denominator_prod
            
            numerator_prod = params['theta'][idx] * numerator_prod
            denominator_prod = params['theta'][observed].T * denominator_prod
            denominator_prod = np.sum(denominator_prod)
            
            pattern_n = params['pattern_ns'][n][tuple(np.array(idx)[np.where(params['binary_patterns'][n] == 0)[0]])]
            try:
                params['ns'][n][idx] = pattern_n * (numerator_prod / denominator_prod)
            except FloatingPointError:
                params['ns'][n][idx] = 0
    
    return params

In [55]:
def paramsUnchanged(params, old_params, epsilon):
    if np.linalg.norm(params['theta'].flatten() - old_params['theta'].flatten()) > epsilon:
        #print(np.linalg.norm(params['theta'].flatten() - old_params['theta'].flatten()))
        return False
    for subseq_len in params['phis']:
        for subseq in params['phis'][subseq_len]:
            if np.linalg.norm(params['phis'][subseq_len][subseq].flatten() - old_params['phis'][subseq_len][subseq].flatten()) > epsilon:
                #print(np.linalg.norm(params['phis'][subseq_len][subseq].flatten() - old_params['phis'][subseq_len][subseq].flatten()))
                return False
    return True

def doEM(T, params, old_params, epsilon):
    i = 1
    while (i == 1 or not paramsUnchanged(params, old_params, epsilon)) and i < 15000:
        print('Loop', i)
        i += 1
        old_params = copy.deepcopy(params)
        params = EStep(params)
        params = MStep(T, params)
    
    return params

In [56]:
def main():
    final_thetas = list()
    for l in range(100):
        print('Experiment Number', l+1)
        n = 100
        T = 3
        R_scale = 1
        numCats = np.array([2,2,2])
        data = generateBinaryData(n,T, R_scale)
        numPatterns = 2**T
        epsilon = .0005
        start = datetime.datetime.now()
        params, old_params = initParams(data, numCats[:T], numPatterns, n, T)
        params = doEM(T, params, old_params, epsilon)
        final_thetas.append(params['theta'])
    np.save('./fixed_theta_EM_' + str(n), np.array(final_thetas))
    print('Done!')

In [57]:
np.random.seed(12345)
main()

Experiment Number 1
78
Loop 1
Loop 2
Loop 3
Loop 4
Loop 5
Loop 6
Loop 7
Loop 8
Loop 9
Loop 10
Loop 11
Loop 12
Loop 13
Loop 14
Loop 15
Loop 16
Loop 17
Loop 18
Loop 19
Loop 20
Loop 21
Loop 22
Loop 23
Loop 24
Loop 25
Loop 26
Loop 27
Loop 28
Loop 29
Loop 30
Loop 31
Loop 32
Loop 33
Loop 34
Loop 35
Loop 36
Loop 37
Loop 38
Loop 39
Loop 40
Loop 41
Loop 42
Loop 43
Loop 44
Loop 45
Loop 46
Loop 47
Loop 48
Loop 49
Loop 50
Loop 51
Loop 52
Loop 53
Loop 54
Loop 55
Loop 56
Loop 57
Loop 58
Loop 59
Loop 60
Loop 61
Loop 62
Loop 63
Loop 64
Loop 65
Loop 66
Loop 67
Loop 68
Loop 69
Loop 70
Loop 71
Loop 72
Loop 73
Loop 74
Loop 75
Loop 76
Loop 77
Loop 78
Loop 79
Loop 80
Loop 81
Loop 82
Loop 83
Loop 84
Loop 85
Loop 86
Loop 87
Loop 88
Loop 89
Loop 90
Loop 91
Loop 92
Loop 93
Loop 94
Loop 95
Loop 96
Loop 97
Loop 98
Loop 99
Loop 100
Loop 101
Loop 102
Loop 103
Loop 104
Loop 105
Loop 106
Loop 107
Loop 108
Loop 109
Loop 110
Loop 111
Loop 112
Loop 113
Loop 114
Loop 115
Loop 116
Loop 117
Loop 118
Loop 119
Loop 120
Loop 

Loop 79
Loop 80
Loop 81
Loop 82
Loop 83
Loop 84
Loop 85
Loop 86
Loop 87
Loop 88
Loop 89
Loop 90
Loop 91
Loop 92
Loop 93
Loop 94
Loop 95
Loop 96
Loop 97
Loop 98
Loop 99
Loop 100
Loop 101
Loop 102
Loop 103
Loop 104
Loop 105
Loop 106
Loop 107
Loop 108
Loop 109
Loop 110
Loop 111
Loop 112
Loop 113
Loop 114
Loop 115
Loop 116
Loop 117
Loop 118
Loop 119
Loop 120
Loop 121
Loop 122
Loop 123
Loop 124
Loop 125
Loop 126
Loop 127
Loop 128
Loop 129
Loop 130
Loop 131
Loop 132
Loop 133
Loop 134
Loop 135
Loop 136
Loop 137
Loop 138
Loop 139
Loop 140
Loop 141
Loop 142
Loop 143
Loop 144
Loop 145
Loop 146
Loop 147
Loop 148
Loop 149
Loop 150
Loop 151
Loop 152
Loop 153
Loop 154
Loop 155
Loop 156
Loop 157
Loop 158
Loop 159
Loop 160
Loop 161
Loop 162
Loop 163
Loop 164
Loop 165
Loop 166
Loop 167
Loop 168
Loop 169
Loop 170
Loop 171
Loop 172
Loop 173
Loop 174
Loop 175
Loop 176
Loop 177
Loop 178
Loop 179
Loop 180
Loop 181
Loop 182
Loop 183
Loop 184
Loop 185
Loop 186
Loop 187
Loop 188
Loop 189
Loop 190
Loop 191
Loop

Loop 21
Loop 22
Loop 23
Loop 24
Loop 25
Loop 26
Loop 27
Loop 28
Loop 29
Loop 30
Loop 31
Loop 32
Loop 33
Loop 34
Loop 35
Loop 36
Loop 37
Loop 38
Loop 39
Loop 40
Loop 41
Loop 42
Loop 43
Loop 44
Loop 45
Loop 46
Loop 47
Loop 48
Loop 49
Loop 50
Loop 51
Loop 52
Loop 53
Loop 54
Loop 55
Loop 56
Loop 57
Loop 58
Loop 59
Loop 60
Loop 61
Loop 62
Loop 63
Loop 64
Loop 65
Loop 66
Loop 67
Loop 68
Loop 69
Loop 70
Loop 71
Loop 72
Loop 73
Loop 74
Loop 75
Loop 76
Loop 77
Loop 78
Loop 79
Loop 80
Loop 81
Loop 82
Loop 83
Loop 84
Loop 85
Loop 86
Loop 87
Loop 88
Loop 89
Loop 90
Loop 91
Loop 92
Loop 93
Loop 94
Loop 95
Loop 96
Loop 97
Loop 98
Loop 99
Loop 100
Loop 101
Loop 102
Loop 103
Loop 104
Loop 105
Loop 106
Loop 107
Loop 108
Loop 109
Loop 110
Loop 111
Loop 112
Loop 113
Loop 114
Loop 115
Loop 116
Loop 117
Loop 118
Loop 119
Loop 120
Loop 121
Loop 122
Loop 123
Loop 124
Loop 125
Loop 126
Loop 127
Loop 128
Loop 129
Loop 130
Loop 131
Loop 132
Loop 133
Loop 134
Loop 135
Loop 136
Loop 137
Loop 138
Loop 139
Loop 140

Loop 956
Loop 957
Loop 958
Loop 959
Loop 960
Loop 961
Loop 962
Loop 963
Loop 964
Loop 965
Loop 966
Loop 967
Loop 968
Loop 969
Loop 970
Loop 971
Loop 972
Loop 973
Loop 974
Loop 975
Loop 976
Loop 977
Loop 978
Loop 979
Loop 980
Loop 981
Loop 982
Loop 983
Loop 984
Loop 985
Loop 986
Loop 987
Loop 988
Loop 989
Loop 990
Loop 991
Loop 992
Loop 993
Loop 994
Loop 995
Loop 996
Loop 997
Loop 998
Loop 999
Loop 1000
Loop 1001
Loop 1002
Loop 1003
Loop 1004
Loop 1005
Loop 1006
Loop 1007
Loop 1008
Loop 1009
Loop 1010
Loop 1011
Loop 1012
Loop 1013
Loop 1014
Loop 1015
Loop 1016
Loop 1017
Loop 1018
Loop 1019
Loop 1020
Loop 1021
Loop 1022
Loop 1023
Loop 1024
Loop 1025
Loop 1026
Loop 1027
Loop 1028
Loop 1029
Loop 1030
Loop 1031
Loop 1032
Loop 1033
Loop 1034
Loop 1035
Loop 1036
Loop 1037
Loop 1038
Loop 1039
Loop 1040
Loop 1041
Loop 1042
Loop 1043
Loop 1044
Loop 1045
Loop 1046
Loop 1047
Loop 1048
Loop 1049
Loop 1050
Loop 1051
Loop 1052
Loop 1053
Loop 1054
Loop 1055
Loop 1056
Loop 1057
Loop 1058
Loop 1059
Loop

Loop 1917
Loop 1918
Loop 1919
Loop 1920
Loop 1921
Loop 1922
Loop 1923
Loop 1924
Loop 1925
Loop 1926
Loop 1927
Loop 1928
Loop 1929
Loop 1930
Loop 1931
Loop 1932
Loop 1933
Loop 1934
Loop 1935
Loop 1936
Loop 1937
Loop 1938
Loop 1939
Loop 1940
Loop 1941
Loop 1942
Loop 1943
Loop 1944
Loop 1945
Loop 1946
Loop 1947
Loop 1948
Loop 1949
Loop 1950
Loop 1951
Loop 1952
Loop 1953
Loop 1954
Loop 1955
Loop 1956
Loop 1957
Loop 1958
Loop 1959
Loop 1960
Loop 1961
Loop 1962
Loop 1963
Loop 1964
Loop 1965
Loop 1966
Loop 1967
Loop 1968
Loop 1969
Loop 1970
Loop 1971
Loop 1972
Loop 1973
Loop 1974
Loop 1975
Loop 1976
Loop 1977
Loop 1978
Loop 1979
Loop 1980
Loop 1981
Loop 1982
Loop 1983
Loop 1984
Loop 1985
Loop 1986
Loop 1987
Loop 1988
Loop 1989
Loop 1990
Loop 1991
Loop 1992
Loop 1993
Loop 1994
Loop 1995
Loop 1996
Loop 1997
Loop 1998
Loop 1999
Loop 2000
Loop 2001
Loop 2002
Loop 2003
Loop 2004
Loop 2005
Loop 2006
Loop 2007
Loop 2008
Loop 2009
Loop 2010
Loop 2011
Loop 2012
Loop 2013
Loop 2014
Loop 2015
Loop 2016


...

Loop 11388
Loop 11389
Loop 11390
Loop 11391
Loop 11392
Loop 11393
Loop 11394
Loop 11395
Loop 11396
Loop 11397
Loop 11398
Loop 11399
Loop 11400
Loop 11401
Loop 11402
Loop 11403
Loop 11404
Loop 11405
Loop 11406
Loop 11407
Loop 11408
Loop 11409
Loop 11410
Loop 11411
Loop 11412
Loop 11413
Loop 11414
Loop 11415
Loop 11416
Loop 11417
Loop 11418
Loop 11419
Loop 11420
Loop 11421
Loop 11422
Loop 11423
Loop 11424
Loop 11425
Loop 11426
Loop 11427
Loop 11428
Loop 11429
Loop 11430
Loop 11431
Loop 11432
Loop 11433
Loop 11434
Loop 11435
Loop 11436
Loop 11437
Loop 11438
Loop 11439
Loop 11440
Loop 11441
Loop 11442
Loop 11443
Loop 11444
Loop 11445
Loop 11446
Loop 11447
Loop 11448
Loop 11449
Loop 11450
Loop 11451
Loop 11452
Loop 11453
Loop 11454
Loop 11455
Loop 11456
Loop 11457
Loop 11458
Loop 11459
Loop 11460
Loop 11461
Loop 11462
Loop 11463
Loop 11464
Loop 11465
Loop 11466
Loop 11467
Loop 11468
Loop 11469
Loop 11470
Loop 11471
Loop 11472
Loop 11473
Loop 11474
Loop 11475
Loop 11476
Loop 11477
Loop 11478

Loop 12251
Loop 12252
Loop 12253
Loop 12254
Loop 12255
Loop 12256
Loop 12257
Loop 12258
Loop 12259
Loop 12260
Loop 12261
Loop 12262
Loop 12263
Loop 12264
Loop 12265
Loop 12266
Loop 12267
Loop 12268
Loop 12269
Loop 12270
Loop 12271
Loop 12272
Loop 12273
Loop 12274
Loop 12275
Loop 12276
Loop 12277
Loop 12278
Loop 12279
Loop 12280
Loop 12281
Loop 12282
Loop 12283
Loop 12284
Loop 12285
Loop 12286
Loop 12287
Loop 12288
Loop 12289
Loop 12290
Loop 12291
Loop 12292
Loop 12293
Loop 12294
Loop 12295
Loop 12296
Loop 12297
Loop 12298
Loop 12299
Loop 12300
Loop 12301
Loop 12302
Loop 12303
Loop 12304
Loop 12305
Loop 12306
Loop 12307
Loop 12308
Loop 12309
Loop 12310
Loop 12311
Loop 12312
Loop 12313
Loop 12314
Loop 12315
Loop 12316
Loop 12317
Loop 12318
Loop 12319
Loop 12320
Loop 12321
Loop 12322
Loop 12323
Loop 12324
Loop 12325
Loop 12326
Loop 12327
Loop 12328
Loop 12329
Loop 12330
Loop 12331
Loop 12332
Loop 12333
Loop 12334
Loop 12335
Loop 12336
Loop 12337
Loop 12338
Loop 12339
Loop 12340
Loop 12341

Loop 13203
Loop 13204
Loop 13205
Loop 13206
Loop 13207
Loop 13208
Loop 13209
Loop 13210
Loop 13211
Loop 13212
Loop 13213
Loop 13214
Loop 13215
Loop 13216
Loop 13217
Loop 13218
Loop 13219
Loop 13220
Loop 13221
Loop 13222
Loop 13223
Loop 13224
Loop 13225
Loop 13226
Loop 13227
Loop 13228
Loop 13229
Loop 13230
Loop 13231
Loop 13232
Loop 13233
Loop 13234
Loop 13235
Loop 13236
Loop 13237
Loop 13238
Loop 13239
Loop 13240
Loop 13241
Loop 13242
Loop 13243
Loop 13244
Loop 13245
Loop 13246
Loop 13247
Loop 13248
Loop 13249
Loop 13250
Loop 13251
Loop 13252
Loop 13253
Loop 13254
Loop 13255
Loop 13256
Loop 13257
Loop 13258
Loop 13259
Loop 13260
Loop 13261
Loop 13262
Loop 13263
Loop 13264
Loop 13265
Loop 13266
Loop 13267
Loop 13268
Loop 13269
Loop 13270
Loop 13271
Loop 13272
Loop 13273
Loop 13274
Loop 13275
Loop 13276
Loop 13277
Loop 13278
Loop 13279
Loop 13280
Loop 13281
Loop 13282
Loop 13283
Loop 13284
Loop 13285
Loop 13286
Loop 13287
Loop 13288
Loop 13289
Loop 13290
Loop 13291
Loop 13292
Loop 13293

Loop 14137
Loop 14138
Loop 14139
Loop 14140
Loop 14141
Loop 14142
Loop 14143
Loop 14144
Loop 14145
Loop 14146
Loop 14147
Loop 14148
Loop 14149
Loop 14150
Loop 14151
Loop 14152
Loop 14153
Loop 14154
Loop 14155
Loop 14156
Loop 14157
Loop 14158
Loop 14159
Loop 14160
Loop 14161
Loop 14162
Loop 14163
Loop 14164
Loop 14165
Loop 14166
Loop 14167
Loop 14168
Loop 14169
Loop 14170
Loop 14171
Loop 14172
Loop 14173
Loop 14174
Loop 14175
Loop 14176
Loop 14177
Loop 14178
Loop 14179
Loop 14180
Loop 14181
Loop 14182
Loop 14183
Loop 14184
Loop 14185
Loop 14186
Loop 14187
Loop 14188
Loop 14189
Loop 14190
Loop 14191
Loop 14192
Loop 14193
Loop 14194
Loop 14195
Loop 14196
Loop 14197
Loop 14198
Loop 14199
Loop 14200
Loop 14201
Loop 14202
Loop 14203
Loop 14204
Loop 14205
Loop 14206
Loop 14207
Loop 14208
Loop 14209
Loop 14210
Loop 14211
Loop 14212
Loop 14213
Loop 14214
Loop 14215
Loop 14216
Loop 14217
Loop 14218
Loop 14219
Loop 14220
Loop 14221
Loop 14222
Loop 14223
Loop 14224
Loop 14225
Loop 14226
Loop 14227

Loop 3
Loop 4
Loop 5
Loop 6
Loop 7
Loop 8
Loop 9
Loop 10
Loop 11
Loop 12
Loop 13
Loop 14
Loop 15
Loop 16
Loop 17
Loop 18
Loop 19
Experiment Number 84
68
Loop 1
Loop 2
Loop 3
Loop 4
Loop 5
Loop 6
Loop 7
Loop 8
Loop 9
Loop 10
Loop 11
Loop 12
Loop 13
Loop 14
Loop 15
Loop 16
Loop 17
Loop 18
Loop 19
Loop 20
Loop 21
Loop 22
Loop 23
Loop 24
Loop 25
Loop 26
Loop 27
Loop 28
Loop 29
Loop 30
Loop 31
Loop 32
Loop 33
Loop 34
Loop 35
Loop 36
Experiment Number 85
75
Loop 1
Loop 2
Loop 3
Loop 4
Loop 5
Loop 6
Loop 7
Loop 8
Loop 9
Loop 10
Loop 11
Loop 12
Experiment Number 86
56
Loop 1
Loop 2
Loop 3
Loop 4
Loop 5
Loop 6
Loop 7
Loop 8
Loop 9
Loop 10
Loop 11
Loop 12
Loop 13
Loop 14
Loop 15
Loop 16
Loop 17
Loop 18
Loop 19
Loop 20
Loop 21
Loop 22
Loop 23
Loop 24
Loop 25
Loop 26
Loop 27
Loop 28
Loop 29
Loop 30
Loop 31
Loop 32
Loop 33
Experiment Number 87
98
Loop 1
Loop 2
Loop 3
Loop 4
Experiment Number 88
67
Loop 1
Loop 2
Loop 3
Loop 4
Loop 5
Loop 6
Loop 7
Loop 8
Loop 9
Loop 10
Loop 11
Loop 12
Loop 13
Loop 14

In [58]:
fnames = ['./fixed_theta_EM_100.npy', './fixed_theta_EM_1000.npy', './fixed_theta_EM_10000.npy']
numCats = np.array([2,2,2])
gt_theta = np.ones(numCats) * (1/8)
norms = []
for file in fnames:
    norms.append(list())
    theta = np.load(file)
    non_nan = []
    for i in range(theta.shape[0]):
        if not np.isnan(theta[i]).any():
            non_nan.append(i)
    print(len(non_nan))
    for val in non_nan:
        norms[-1].append(np.linalg.norm(theta[val] - gt_theta))
    print('mean', np.mean(norms[-1]))
    print('var', np.var(norms[-1]))

100
mean 0.157546327573
var 0.00211331679673
100
mean 0.0648820135542
var 0.00123155407848
100
mean 0.0435808446308
var 0.000567042289083
